# Assignment 2: Milestone I Natural Language Processing
## Task 2&3
#### Student Name: Sai Vamsi Chunduru
#### Student ID: s3884753

Date: 02-10-2021

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* os
* gensim.matutils.sparse2full
* CountVectorizer
* FastText
* numpy
* train_test_split
* LogisticRegression
* Dictionary
* TfidfModel
* gensim.downloader as api

## Introduction
Abrief information of this assessment task here.

## Task 2
I generated a different types of feature representations for the collection of job advertisements.In this task, we will only consider the description of the job advertisement.

Bag-of-words model: Generated the Count vector representation for each job advertisement description.The generated Count vector
representation is based on the generated vocabulary in Task 1 (as saved in vocab.txt).

Models based on word embeddings:
we generate feature representation of job advertisement description based on
the following language models, respectively:
■ FastText language model trained based on the provided job advertisement
descriptions, with a 200 embedding dimension.
■ 2 pre-trained language models- GoogleNews300 and Glove.

For each of the above mentioned language models, I built the weighted (i.e., TF-IDF weighted) and unweighted vector representation for each job advertisement

count_vectors.txt- This file stores the sparse count vector representation of job advertisement descriptions in the required format. 

## Task 3

In this task, we build machine learning models for classifying the category of a job advertisement text.

A simple model that can be considered is the logistic regression model from sklearn.

I conducted two sets of experiments on the provided dataset to investigate the following two questions, respectively.

Q1: Language model comparison : Which language model we built based on job advertisement descriptions performs the best with
the chosen machine learning model.

To answer these questions, we build machine learning models based on the feature representations of the documents.We created 7 different models- 1 model based on count vector representations, 2 models of FastText(unweighted and weighted tfidf), 2 models of GoogleNews300(unweighted and weighted tfidf), and 2 models of Glove(unweighted and weighted tfidf). 
We evaluated the various model performance based on their model scores.

Q2: Does more information provide higher accuracy?Will adding extra information help to boost up the accuracy of the model? To answer this question,

In this experiment we explore other features of a job advertisement, e.g., the title of the job
position. and we looked to answer this question.
So to answer the above question I considered 2 additional data sets like job title + escription and job title data alone from job advertisement datasets. I built a FastText unweighted model based and compared the performance of classification models with the FastText model built on job description data alone.
I Considered unweighted FastText model to comare these 3 different (job description, job title+ description and job title) classification models. 

  Reasons for considering Unweighted FastText model: This model displayed much higher accuracy score along with weighted fastText model. 
This is expected cause we bulit the FastText model based  on our processed data rather than on the pre trained models.
Eventhough weighted FastText model accuracy is higher than unweighted Fasttext model.We used unweighted FastText model due it's minimal computational resource usage.
I used different solvers (sag and saga) of logistic regression to compare the model performances, but the default solver "ibfgs" performs better than others. So we used ibfgs.

## Importing libraries 

In [1]:
# Code to import libraries that we need in this assessment.
import pandas as pd
import os
import numpy as np
from gensim.matutils import sparse2full
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.fasttext import FastText
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel
import gensim.downloader as api


## Task 2. Generating Feature Representations for Job Advertisement Descriptions

...... Sections and code blocks on buidling different document feature represetations


<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

In [6]:
#readinf the job_ads1 csv file this is saved in task-1, used here extract the data ina data frame structure
job_ads_df=pd.read_csv("job_ads1.csv")

In [7]:
#Displaying the 4 columns of dataframe.
job_ads_df.head(4)

,job title,webindex,category,Job Description processed,Job title Description processed,Job title processed
0,Plant Engineer,62119057,Engineering,established manufacturer supplier quality wate...,plant engineer established manufacturer suppli...,plant
1,Residential Care Worker,66314490,Healthcare_Nursing,timeout children homes rapidly expanding foref...,residential care worker timeout children homes...,residential
2,CHEF DE RANG FOR MICHELIN STARRED RESTAURANT,69182387,Hospitality_Catering,french restaurant club gascon michelin establi...,chef de rang michelin starred restaurant frenc...,rang michelin starred
3,Inpatient Ward Team Leader,64752715,Healthcare_Nursing,inpatient ward leader description patient lead...,inpatient ward leader inpatient ward leader de...,inpatient ward leader


In [8]:
#Validating the length of categories.
len(job_ads_df["category"])

55449

In [9]:
#Storing the categories in a list structure.
category12=job_ads_df["category"]

In [10]:
#Checking for null values in category
job_ads_df["category"].isna().sum()

0

In [7]:
#storing the job title-unprocessed in list format.
job_title=job_ads_df["job title"]

In [8]:
#storing the job description-processed in list format
jobads12=job_ads_df['Job Description processed']

In [9]:
#splitting the job description into token and saving them in a list format
job_desc_tokens=[i.split(" ") for i in jobads12]

In [10]:
#storing the job title + description-processed in list format
jobads13=job_ads_df['Job title Description processed']

In [11]:
#splitting the job title+ description into token and saving them in a list format
job_title_desc_tokens=[i.split(" ") for i in jobads13]

In [12]:
#storing the job title -processed in list format
job_title_tokens=job_ads_df['Job title processed'].astype(str)

In [13]:
#Validating the data type of elements
type(job_title_tokens[2])

str

In [14]:
#storing the job webindex in list format
webindex12=job_ads_df['webindex']

In [15]:
#Reading the vocabulary from vocb.txt-from task 1
voc1=[]
with open('vocab.txt') as f:
    for line in f:
        voc1.append(line)

In [16]:
#Displaying the sizes of job advertisements data, vocbulary and webindex.
print(len(jobads12))
print(len(voc1))
print(len(webindex12))

55449
40038
55449


In [17]:
#Displaying the sample data present in jobads-description,vocab and webindex
print(jobads12[0])
print(voc1[0])
print(webindex12[0])

established manufacturer supplier quality water treatment plants ranging basic water softeners reverse osmosis equipment customer complex water treatment solutions meet clients requirements flexibility tailoring product budgets due expansion increased workload seeking recruit planet engineer cover accounts corridor responsibilities include conducting routine sampling analysis water systems interpreting results maintenance installation chemical dosing systems servicing accounts industrial commercial industries complete accordance approved code practice ideal applicant minimum years relevant industry reverse osmosis water softeners water filters uv equipment full driving license return offering competitive package ideal
a'level:0

62119057


In [18]:
#converting webindex data type to string
webindex=[str(i) for i in webindex12]

In [19]:
#splitting and saving the extracted vocab in a list format.
vocab1=[i.split(":")[0] for i in voc1]

In [20]:
#Displaying sample vocabs
vocab1[0:5]

["a'level", 'aa', 'aaa', 'aaappointments', 'aab']

In [21]:
# Code to perform the task...
#Creating count vector representation
cVectorizer = CountVectorizer(analyzer = "word",vocabulary = vocab1) # initialised the CountVectorizer

In [22]:
count_features = cVectorizer.fit_transform([' '.join(desc) for desc in job_desc_tokens])
# generate the count vector representation for all words in job description
print(count_features.shape)

(55449, 40038)


In [23]:
#created a count_vec_list1 to hold webindex and count features data.
count_vec_list1=[]
strz12=''
for i in count_features:
    t1=i.toarray()
    t2=t1[0]
    for word_index, value in zip(range(0, len(vocab1)), t2): # using word_index:value format
        if value > 0:
           strz12=strz12+str(word_index)+":"+str(value)+" "  
    count_vec_list1.append(strz12)
    strz12=''

In [24]:
#Displaying the length of extracted count_vec_list1 list.
len(count_vec_list1)

55449

In [25]:
#Replacing the spaces in the spaces in count_vec_list1 with commas(,) as per requirements.
count_vec=[i.replace(" ", ",")[0:-1] for i in count_vec_list1]

In [26]:
def gen_docVecs_modified1(wv,tk_txts): # generate vector representation for documents
    docs_vectors =[] # creating empty final list
    for i in range(0,len(tk_txts)):
            tokens = tk_txts[i]
            temp = []  # creating a temporary list(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
            temp2=[]   #It holds the temp values after each iteration
            for w_ind in range(0, len(tokens)): # looping through each word of a single document and spliting through space
                try:
                    word = tokens[w_ind]
                    word_vec = wv[word] # if word is present in embeddings(google provides weights associate with words(300)/fasttext with size 200) then proceed
                    temp = pd.Series(word_vec) # if word is present then assign it to temp.
                    temp2.append(temp)
                except:
                    pass
                temp=[]
            zk=np.sum(np.array(temp2),axis=0) #The sum is calculated and stored in zk, a temp variable
            docs_vectors.append(zk) #toring all the doc vectors in this list.
    return docs_vectors

In [27]:
# Code to perform the task...
#Generating document vectors with FastText model.
# 1. Set the corpus file names/path
corpus_file = './job_descrip1.txt'

# 2. Initialise the Fast Text model
jobDescFT = FastText(vector_size=200) 

# 3. build the vocabulary
jobDescFT.build_vocab(corpus_file=corpus_file)

# 4. train the model
jobDescFT.train(
    corpus_file=corpus_file, epochs=jobDescFT.epochs,
    total_examples=jobDescFT.corpus_count, total_words=jobDescFT.corpus_total_words,
)

print(jobDescFT)

FastText(vocab=22816, vector_size=200, alpha=0.025)


In [28]:
#Displaying the job description word vectors.
jobDescFT_wv = jobDescFT.wv
print(jobDescFT_wv)

In [29]:
# Save the FastText model-can be used directly wthout building amodel.
jobDescFT.save("jobDescFT.model")

In [30]:
# loading the trained Fasttext model based on bbc News data
#We can load this model or can comment this whole block,
jobDescFT = FastText.load("jobDescFT.model")
print(jobDescFT)
jobDescFT_wv= jobDescFT.wv

FastText(vocab=22816, vector_size=200, alpha=0.025)


In [31]:
# NOTE this takes some time to finish running
# generate document embeddings.
jobDescFT_dvs12 = gen_docVecs_modified1(jobDescFT_wv,job_desc_tokens)
jobDescFT_dvs_arr=np.array(jobDescFT_dvs12) #Converting the generated document vectors into numpy arrays.
print(jobDescFT_dvs_arr.shape)  # displaying the shape of array


(55449, 200)


In [35]:
#Imported gensim loader to load googlenews 300 pre trained model.
import gensim.downloader as api
preTW2v_g300_wv = api.load('word2vec-google-news-300')

In [36]:
#checking the vector size of extracted google 300-news, word vector
preTW2v_g300_wv.vector_size

300

In [37]:
# NOTE this can take some time to finish running
# generate document embeddings
jobDesc_g300_dvs12 = gen_docVecs_modified1(preTW2v_g300_wv,job_desc_tokens)
jobDesc_g300_dvs_arr=np.array(jobDesc_g300_dvs12)  #Converting the generated document vectors into numpy arrays.
print(jobDesc_g300_dvs_arr.shape) # displaying the shape of array


(55449, 300)


In [40]:
#generating word vectors based on pre trained model-golve
path_to_glove_file = os.path.join("glove/glove.6B.200d.txt") # specified the path to the embedding file

In [41]:
glove_wv = {} # initialise an empty dcitionary
with open(path_to_glove_file, encoding="utf-8") as f: # open the txt file containing the word embedding vectors
    for line in f:
        word, coefs = line.split(maxsplit=1) # The maxsplit defines the maximum number of splits. 
                                             # in the above example, it will give:
                                             # ['population','0.035182 1.4248 0.9758 0.1313 -0.66877 0.8539 -0.11525 ......']
        coefs = np.fromstring(coefs, "f", sep=" ") # construct an numpy array from the string 'coefs', 
                                                   # e.g., '0.035182 1.4248 0.9758 0.1313 -0.66877 0.8539 -0.11525 ......'
        glove_wv[word] = coefs # create the word and embedding vector mapping

print("Found %s word vectors." % len(glove_wv))

Found 400000 word vectors.


In [43]:
# NOTE this can take some time to finish running
# generate document embeddings-unweighted
jobDesc_glove_dvs12 = gen_docVecs_modified1(glove_wv,job_desc_tokens)

In [44]:
jobDesc_glove_dvs_arr=np.array(jobDesc_glove_dvs12) #Converting the generated document vectors into numpy arrays.
jobDesc_glove_dvs_arr.shape # displaying the shape of array

(55449, 200)

In [45]:
#calculating the tfidf vecors
from sklearn.feature_extraction.text import TfidfVectorizer
tVectorizer = TfidfVectorizer(analyzer = "word",vocabulary = vocab1) # initialised the TfidfVectorizer
tfidf_features = tVectorizer.fit_transform([' '.join(desc) for desc in job_desc_tokens]) # generate the tfidf vector representation for all word in job description
tfidf_features.shape #displaying the shape of extracted tfidf_features

(55449, 40038)

In [46]:
#formatting the vocabulary in the required dictonary form.
def gen_vocIndex(voc1):
    voc_Ind = [l.split(':') for l in voc1] # each line is 'index,word'
    return {int(vi[1]):vi[0] for vi in voc_Ind}

In [47]:
# Generates the w_index:word dictionary
voc_dict = gen_vocIndex(voc1)
voc_dict

{0: "a'level",
 1: 'aa',
 2: 'aaa',
 3: 'aaappointments',
 4: 'aab',
 5: 'aac',
 6: 'aacc',
 7: 'aae',
 8: 'aah',
 9: 'aak',
 10: 'aamm',
 11: 'aan',
 12: 'aand',
 13: 'aantrekkelijk',
 14: 'aap',
 15: 'aar',
 16: 'aarca',
 17: 'aardman',
 18: 'aareon',
 19: 'aaron',
 20: 'aaronwallis',
 21: 'aarosette',
 22: 'aarosettehotel',
 23: 'aarosettelivein',
 24: 'aarosetterestaurant',
 25: 'aarosetterestaurantlondon',
 26: 'aarosettes',
 27: 'aarosettesinternationalchainofhotels',
 28: 'aarosetteslivein',
 29: 'aasl',
 30: 'aasp',
 31: 'aastra',
 32: 'aat',
 33: 'aatom',
 34: 'ab',
 35: 'aba',
 36: 'abacus',
 37: 'abaility',
 38: 'abandoned',
 39: 'abandonment',
 40: 'abandons',
 41: 'abap',
 42: 'abaqus',
 43: 'abatement',
 44: 'abb',
 45: 'abbas',
 46: 'abbey',
 47: 'abbeywood',
 48: 'abbie',
 49: 'abbot',
 50: 'abbots',
 51: 'abbott',
 52: 'abby',
 53: 'abbyy',
 54: 'abc',
 55: 'abd',
 56: 'abdo',
 57: 'abdominal',
 58: 'abdul',
 59: 'abdulla',
 60: 'abel',
 61: 'abellprotocoleducation',
 

In [48]:
#Displaying the size of vocab dictonary
len(voc_dict)

40038

In [49]:
#Checking the type of tfidf_features
type(tfidf_features)

scipy.sparse.csr.csr_matrix

In [50]:
#reducing the float size from float32 to float16 to overome memory issues.
tfidf_features1=tfidf_features.astype("float16")

In [51]:
#displaying a sample from tfidf features.
tfidf_features1[0]

<1x40038 sparse matrix of type '<class 'numpy.float16'>'
	with 71 stored elements in Compressed Sparse Row format>

In [52]:
def write_vectorFile(data_features,filename):
    num = data_features.shape[0] # the number of document
    out_file = open(filename, 'w') # creates a txt file and open to save the vector representation
    for a_ind in range(0, num): # loop through each article by index
        for f_ind in data_features[a_ind].nonzero()[1]: # for each word index that has non-zero entry in the data_feature
            value = data_features[a_ind][0,f_ind] # retrieve the value of the entry from data_features
            out_file.write("{}:{} ".format(f_ind,value)) # write the entry to the file in the format of word_index:value
        out_file.write('\n') # start a new line after each article
    out_file.close() # close the file

In [53]:
tVector_file = "./jobDesc_tVector.txt" # file name of the tfidf vector
write_vectorFile(tfidf_features,tVector_file) # write the tfidf vector to file. So the tVector file can be used again the below tasks at later stages.

In [54]:
def doc_wordweights(fName_tVectors, voc_dict):
    tfidf_weights = [] # a list to store the  word:weight dictionaries of documents
    
    with open(fName_tVectors) as tVecf: 
        tVectors = tVecf.read().splitlines() # each line is a tfidf vector representation of a document in string format 'word_index:weight word_index:weight .......'
    for tv in tVectors: # for each tfidf document vector
        tv = tv.strip()
        weights = tv.split(' ') # list of 'word_index:weight' entries
        weights = [w.split(':') for w in weights] # change the format of weight to a list of '[word_index,weight]' entries
        wordweight_dict = {voc_dict[int(w[0])]:w[1] for w in weights} # construct the weight dictionary, where each entry is 'word:weight'
        tfidf_weights.append(wordweight_dict) 
    return tfidf_weights

fName_tVectors = 'jobDesc_tVector.txt'
tfidf_weights = doc_wordweights(fName_tVectors, voc_dict)

In [55]:
#Displaying the length of sample elements
len(tfidf_weights[0])

71

In [56]:
# extended version of the `gen_docVecs` function-to be used to create weighted tfidf documents vectors
def gen_docVecs_weighted(wv,tk_txts,tfidf = []): # generate vector representation for documents
    docs_vectors =[] 
    for i in range(0,len(tk_txts)):
        tokens = list(set(tk_txts[i])) # get the list of distinct words of the document
        temp=[] # creating a temp list(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
        temp2=[] #created a temp2 list to append temp values.
        for w_ind in range(0, len(tokens)): # looping through each word of a single document and spliting through space
            try:
                word = tokens[w_ind]
                word_vec = wv[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                
                if tfidf != []:
                    word_weight = float(tfidf[i][word])
                else:
                    word_weight = 1
                #temp = temp.append(pd.Series(word_vec*word_weight), ignore_index = True) # if word is present then append it to temporary dataframe
                temp = pd.Series(word_vec*word_weight)
                temp2.append(temp)
            except:
                pass
            temp=[]
        zk=np.sum(np.array(temp2),axis=0) #adding the array values across axis 0-running vertically dowards across rows.
        docs_vectors.append(zk) #
    return docs_vectors

In [57]:
#Generated a weighted tfidf document vector for glove word vectors
weighted_jobDesc_glove_dvs = gen_docVecs_weighted(glove_wv,job_desc_tokens,tfidf_weights)

In [58]:
weighted_jobDesc_glove_dvs_arr=np.array(weighted_jobDesc_glove_dvs) #Converting the document vectors into arrays.
weighted_jobDesc_glove_dvs_arr.shape #Displaying the size of array

(55449, 200)

In [60]:
#Generated a weighted tfidf document vector for fastText word vectors
weighted_jobDescFT_dvs = gen_docVecs_weighted(jobDescFT_wv,job_desc_tokens,tfidf_weights) 

In [61]:
weighted_jobDescFT_dvs_arr=np.array(weighted_jobDescFT_dvs) #Converting the document vectors into arrays.
weighted_jobDescFT_dvs_arr.shape #Displaying the size of array

(55449, 200)

In [62]:
#Generated a weighted tfidf document vector for google-300-news word vectors
weighted_jobDesc_g300 = gen_docVecs_weighted(preTW2v_g300_wv,job_desc_tokens,tfidf_weights)

In [63]:
weighted_jobDesc_g300_arr=np.array(weighted_jobDesc_g300) #Converting the document vectors into arrays.
weighted_jobDesc_g300_arr.shape #Displaying the size of array

(55449, 300)

### Saving outputs
Save the count vector representation as per spectification.
- count_vectors.txt

In [64]:
# code to save output data...
#creating count_vectors.txt file as per the required format.
with open('count_vectors.txt', 'w') as f:
    for i,j in zip(count_vec,webindex) :
        ss="#"+str(j)+","+i
        f.write("%s\n" % ss)
        

## Task 3. Job Advertisement Classification

...... Sections and code blocks on buidling classification models based on different document feature represetations. 
Detailed comparsions and evaluations on different models to answer each question as per specification. 


In [89]:
list_models=[]  #list to store the classifiation model name
list_models_score=[] #List to store the accuracy score of the respective model.

In [90]:
#Created a logistic regression model with max_iterations of 1000 and solver as default-ibfgs and passing count vectors and categories
seed=0 #seed is declared and initialised.
# creating training and test split
X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(count_features, category12, list(range(0,len(category12))),test_size=0.30, random_state=seed)
jobDescv_LRmodel = LogisticRegression(max_iter = 1000,random_state=seed) #logistic regression model is built..
jobDescv_LRmodel.fit(X_train, y_train)
temp=jobDescv_LRmodel.score(X_test, y_test) #accuracy of count vectors model is extracted
temp  #Displaying the accuracy score

0.8763450556056508

In [3]:
dfk=pd.read_csv("jobDescFT_dvs.csv")

In [4]:
kk1=np.array(dfk) #Converting the document vectors into arrays.


In [5]:
kk1.shape

(55449, 200)

In [13]:
jobDescFT_dvs_arr=kk1

In [91]:
list_models.append("Count Vectors LR model") #stored model name
list_models_score.append(temp) #stored model score

In [11]:
import pickle

In [15]:
pkl_filn="jobDescFT.pkl"
with open(pkl_filn,'wb') as file:
    pickle.dump(jobDesFT_LRmodel,file)

In [14]:
#Created a logistic regression model with max_iterations of 6000 and solver as default-ibfgs and passing unweighted fasttext doc vectors and categories
seed = 0 #seed is declared and initialised.
# creating training and test split
X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(jobDescFT_dvs_arr, category12, list(range(0,len(category12))),test_size=0.30, random_state=seed)
jobDesFT_LRmodel = LogisticRegression(max_iter = 6000,random_state=seed)
jobDesFT_LRmodel.fit(X_train, y_train)
temp=jobDesFT_LRmodel.score(X_test, y_test) #accuracy of unweighted fasttext model is extracted
temp  #displaying score

0.8658851818455064

In [100]:
list_models.append("FastText unweighted LR model") #stored model name
list_models_score.append(temp) #stored model score

In [101]:
#Created a logistic regression model with max_iterations of 5000 and solver as default-ibfgs and passing weighted fasttext doc vectors and categories
seed = 0 #seed is declared and initialzed. It will be used as random state in logistic regression model.
X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(weighted_jobDescFT_dvs_arr, category12, list(range(0,len(category12))),test_size=0.30, random_state=seed)
# creating training and test split
jobDesFT_weighted_LRmodel = LogisticRegression(max_iter = 5000,random_state=seed)
jobDesFT_weighted_LRmodel.fit(X_train, y_train)
temp=jobDesFT_weighted_LRmodel.score(X_test, y_test) #accuracy of weighted fasttext model is extracted
temp #displaying score

0.8696122633002705

In [102]:
list_models.append("FastText weighted tfidf LR model") #stored model name
list_models_score.append(temp) #stored model score

In [103]:
#Created a logistic regression model with max_iterations of 5000 and solver as default-ibfgs and passing unweighted google-news-300 doc vectors and categories
seed = 0 #seed is declared and initialzed. It will be used as random state in logistic regression model.
# creating training and test split
X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(jobDesc_g300_dvs_arr, category12, list(range(0,len(category12))),test_size=0.30, random_state=seed)
jobDesg300_LRmodel = LogisticRegression(max_iter = 5000,random_state=seed)
jobDesg300_LRmodel.fit(X_train, y_train)
temp=jobDesg300_LRmodel.score(X_test, y_test) #accuracy of unweighted google 300 news model is extracted
temp #displaying score

0.8592125037571385

In [104]:
list_models.append("Pre trained google-300-news unweighted LR model") #stored model name
list_models_score.append(temp)  #stored model score

In [105]:
#Created a logistic regression model with max_iterations of 5000 and solver as default-ibfgs and passing weighted google-news-300 tfidf doc vectors and categories
seed = 0 #seed is declared and initialzed. It will be used as random state in logistic regression model.
# creating training and test split
X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(weighted_jobDesc_g300_arr, category12, list(range(0,len(category12))),test_size=0.30, random_state=seed)
jobDesg300_weighted_LRmodel = LogisticRegression(max_iter = 5000,random_state=seed)
jobDesg300_weighted_LRmodel.fit(X_train, y_train)
temp=jobDesg300_weighted_LRmodel.score(X_test, y_test) #accuracy of weighted google 300 news model is extracted
temp #displaying score

0.8651036970243463

In [106]:
list_models.append("Pre trained google-300-news weighted tfidf LR model") #stored model name
list_models_score.append(temp) #stored model score

In [107]:
#Created a logistic regression model with max_iterations of 5000 and solver as default-ibfgs and passing unweighted glove doc vectors and categories
seed = 0 #seed is declared and initialzed. It will be used as random state in logistic regression model.
# creating training and test split
X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(jobDesc_glove_dvs_arr, category12, list(range(0,len(category12))),test_size=0.30, random_state=seed)
jobDesglove_LRmodel = LogisticRegression(max_iter = 5000,random_state=seed)
jobDesglove_LRmodel.fit(X_train, y_train)
temp=jobDesglove_LRmodel.score(X_test, y_test) #accuracy of unweighted glove model is extracted
temp #displaying score

0.8508566275924256

In [108]:
list_models.append("Pre trained glove unweighted LR model") #stored model name
list_models_score.append(temp) #stored model score

In [109]:
#Created a logistic regression model with max_iterations of 5000 and solver as default-ibfgs and passing weighted tfidf glove doc vectors and categories
seed = 0 #seed is declared and initialzed. It will be used as random state in logistic regression model.
# creating training and test split
X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(weighted_jobDesc_glove_dvs_arr, category12, list(range(0,len(category12))),test_size=0.30, random_state=seed)
jobDesglove_weighted_LRmodel = LogisticRegression(max_iter = 5000,random_state=seed)
jobDesglove_weighted_LRmodel.fit(X_train, y_train)
temp=jobDesglove_weighted_LRmodel.score(X_test, y_test) #accuracy of weighted glove model is extracted
temp #displaying score

0.8515779981965735

In [110]:
list_models.append("Pre trained glove weighted tfidf LR model")  #stored model name
list_models_score.append(temp) #stored model score

In [115]:
print("Classification model"+ " "*50 + "Model score")
for i,j in zip(list_models,list_models_score):
    print(i.ljust(70)+str(j))

Classification model                                                  Model score
Count Vectors LR model                                                0.8763450556056508
FastText unweighted LR model                                          0.8659452960625188
FastText weighted tfidf LR model                                      0.8696122633002705
FastText unweighted LR model                                          0.8592125037571385
FastText weighted tfidf LR model                                      0.8651036970243463
Pre trained google-300-news unweighted LR model                       0.8508566275924256
Pre trained google-300-news weighted tfidf LR model                   0.8515779981965735


Based on above model scores, we can conclude that count vector logistic regression model score (0.876) is higher than other models. And only Fastext weighted model got closer score(0.869).

Also, all models creted using the weighted document vectores performed much better than the unweighted ones.

In terms of model performance, count vector model beats the other models in terms of accuracy scores.

Pretrained models, google300-news and glove did not perform well. As fasttext and counter vector generated word vectors based on the inputted datasets whereas the pre trained word vectors are much generalised. Hence, we observe this disparity in pre trained vs models trained on the current data.

Building Logistic regression models of Job title+description and job title fasttext word embeddings. and comparing that with the extracted logistic regression model based on job description data alone.


In [116]:
# Code to perform the task...
#Creating a fast text model based on job Title + description
# 1. Set the corpus file names/path
corpus_file = './job_title_descrip1.txt'

# 2. Initialise the Fast Text model
jobTitleDescFT = FastText(vector_size=200) 

# 3. build the vocabulary
jobTitleDescFT.build_vocab(corpus_file=corpus_file)

# 4. train the model
jobTitleDescFT.train(
    corpus_file=corpus_file, epochs=jobTitleDescFT.epochs,
    total_examples=jobTitleDescFT.corpus_count, total_words=jobTitleDescFT.corpus_total_words,
)

print(jobTitleDescFT) 

FastText(vocab=23115, vector_size=200, alpha=0.025)


In [117]:
jobTitleDescFT_wv = jobTitleDescFT.wv  #extracted word vectors based on the job title+description data.
print(jobTitleDescFT_wv)

In [118]:
# NOTE this can take some time to finish running
# generate docum   ent embeddings
jobTitleDescFT_dvs12 = gen_docVecs_modified1(jobTitleDescFT_wv,job_title_desc_tokens) #extracting the unweighted fasttext doc vectors for job title+description data.
jobTitleDesc_dvs_arr=np.array(jobTitleDescFT_dvs12) #converting extracted doc vectors into arrays.
print(jobTitleDesc_dvs_arr.shape)  #displaying the shape of array.


(55449, 200)


In [119]:
#modelling based on unweighted fasttext- job title+description
#Created a logistic regression model with max_iterations of 6000 and solver as default-ibfgs and passing unweighted fasttext doc vectors of job title+description and categories
seed = 0 #seed is declared and initialzed. It will be used as random state in logistic regression model.
# creating training and test split
X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(jobTitleDesc_dvs_arr,category12, list(range(0,len(category12))),test_size=0.30, random_state=seed)
jobTitleDescFT_LRmodel = LogisticRegression(max_iter = 6000,random_state=seed)
jobTitleDescFT_LRmodel.fit(X_train, y_train)
temp1=jobTitleDescFT_LRmodel.score(X_test, y_test) #accuracy of unweighted fasttext model is extracted
temp1 #displaying the score

0.8694319206492336

In [120]:
# Code to perform the task...
#Creating a fast text model based on job Title 
# 1. Set the corpus file names/path
corpus_file = './job_title1.txt'

# 2. Initialise the Fast Text model
jobTitleFT = FastText(vector_size=200) 

# 3. build the vocabulary
jobTitleFT.build_vocab(corpus_file=corpus_file)

# 4. train the model
jobTitleFT.train(
    corpus_file=corpus_file, epochs=jobTitleFT.epochs,
    total_examples=jobTitleFT.corpus_count, total_words=jobTitleFT.corpus_total_words,
)

print(jobTitleFT)

FastText(vocab=3097, vector_size=200, alpha=0.025)


In [121]:
jobTitleFT_wv = jobTitleFT.wv  #Extracted word vectors based on the job title data.
print(jobTitleFT_wv) 

In [122]:
# NOTE this can take some time to finish running
# generated document embeddings
jobTitleFT_dvs12 = gen_docVecs_modified1(jobTitleFT_wv,job_title_tokens)  #extracted document vectors.
jobTitleFT_dvs_arr=np.array(jobTitleFT_dvs12) #the document vectors are converted into array
print(jobTitleFT_dvs_arr.shape) #Displaying the array


(55449, 200)


In [124]:
#modelling based on unweighted fasttext
#Created a logistic regression model with max_iterations of 1000 and solver as default-ibfgs and passing unweighted fasttext doc vectors of job title and categories
seed = 0 #seed is declared and initialzed. It will be used as random state in logistic regression model.
# creating training and test split
X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(jobTitleFT_dvs_arr,category12, list(range(0,len(category12))),test_size=0.30, random_state=seed)
jobTitleFT_LRmodel = LogisticRegression(max_iter = 1000,random_state=seed)
jobTitleFT_LRmodel.fit(X_train, y_train)
temp2=jobTitleFT_LRmodel.score(X_test, y_test)#accuracy of unweighted fastext logistic regression model is extracted
temp2 #displaying score.

0.34445446348061315

In [127]:
print("Data used to build model"+" "*20+"model score")
print(" Job description".ljust(40)+str(list_models_score[1]))
print(" Job Title+ description".ljust(40)+str(temp1))
print(" Job Title".ljust(40)+str(temp2))

Data used to build model                    model score
 Job description                        0.8659452960625188
 Job Title+ description                 0.8694319206492336
 Job Title                              0.34445446348061315


Based on the above classification model score, we can confirm that the model trained on job title+description got more accuracy score (0.869) than the model based on job description data (0.865).

Thus, we can conclude that more amount of data can improve the model performance. But on downside too much amount of data also leads to usage of high compuatation resources.

Also, the worse performing model is the model biult by using job title data alone (0.344). 

## Summary
#The approximate time it takes to run task 2 and 3 is 140-160 minutes.